In [1]:
import pandas as pd

def clean_numeric(value):
    if isinstance(value, str):
        try:
            return float(value.replace(',', ''))
        except ValueError:
            return value
    return value

# Load CSV files into pandas dataframes
cost_to_host_engineer_location = pd.read_csv('cost_to_host_engineer_location.csv', skipinitialspace=True)
customer_location_reachability = pd.read_csv('customer_location_reachability.csv', skipinitialspace=True)
customer_service_required = pd.read_csv('customer_service_required.csv', skipinitialspace=True)
location_scale_rent = pd.read_csv('location_scale_rent.csv', skipinitialspace=True)
location_to_customer_cost = pd.read_csv('location_to_customer_cost.csv', skipinitialspace=True)
max_num_engineer_hosted = pd.read_csv('max_num_engineer_hosted.csv', skipinitialspace=True)

# Apply the cleaning function to numeric columns only
cost_to_host_engineer_location['Cost for Hosting One Engineer'] = cost_to_host_engineer_location['Cost for Hosting One Engineer'].apply(clean_numeric)
location_scale_rent['Rent'] = location_scale_rent['Rent'].apply(clean_numeric)
numeric_cols = location_to_customer_cost.columns[1:]  # Skip the first column (Customer)
location_to_customer_cost[numeric_cols] = location_to_customer_cost[numeric_cols].applymap(clean_numeric)

# Constant: Number of services an engineer can complete per year
s = 200

# Display each dataframe to ensure they are loaded correctly
print("Cost to Host Engineer Location:\n", cost_to_host_engineer_location.head(), "\n")
print("Customer Location Reachability:\n", customer_location_reachability.head(), "\n")
print("Customer Service Required:\n", customer_service_required.head(), "\n")
print("Location Scale Rent:\n", location_scale_rent.head(), "\n")
print("Location to Customer Cost:\n", location_to_customer_cost.head(), "\n")
print("Max Number of Engineers Hosted:\n", max_num_engineer_hosted.head(), "\n")

# Convert each dataframe into a dictionary to use as parameters in the model

# 1. Cost to Host Engineer in each Location (c_j)
# c_j represents the annual cost for hosting one engineer at location j
c_j = dict(zip(cost_to_host_engineer_location['Location'], cost_to_host_engineer_location['Cost for Hosting One Engineer']))

# 2. Customer Location Reachability (a_ij)
# a_ij represents whether customer i can be served by facility j (1 if possible, 0 otherwise)
a_ij = customer_location_reachability.set_index('Customer').T.to_dict()

# 3. Customer Service Requirement (h_i)
# h_i represents the annual number of services required for customer i
h_i = dict(zip(customer_service_required['Customer'], customer_service_required['Annual # of Services Required']))

# 4. Location Scale Rent (f_jk)
# f_jk represents the annual office rent for facility j at scale k (small, medium, large)
f_jk = location_scale_rent.set_index(['Location', 'Scale'])['Rent'].to_dict()

# 5. Location to Customer Cost (d_ij)
# d_ij represents the cost per service for an engineer to travel between facility j and customer i
d_ij = location_to_customer_cost.set_index('Customer').T.to_dict()

# 6. Maximum Number of Engineers Hosted at a location (m_jk)
# m_jk represents the maximum number of engineers that can be hosted at location j with scale level k (small, medium, large)
m_jk = max_num_engineer_hosted.set_index('Location').T.to_dict()

print("Cost to Host Engineer Dictionary (c_j):\n", c_j, "\n")
print("Reachability Dictionary (a_ij):\n", a_ij, "\n")
print("Customer Service Requirement Dictionary (h_i):\n", h_i, "\n")
print("Location Scale Rent Dictionary (f_jk):\n", f_jk, "\n")
print("Location to Customer Cost Dictionary (d_ij):\n", d_ij, "\n")
print("Max Number of Engineers Hosted Dictionary (m_jk):\n", m_jk, "\n")


Cost to Host Engineer Location:
           Location  Cost for Hosting One Engineer
0         New York                       150000.0
1           Boston                       130000.0
2     Philadelphia                       120000.0
3  Washington D.C.                       140000.0
4        Baltimore                       110000.0 

Customer Location Reachability:
   Customer  New York  Boston  Philadelphia  Washington D.C.  Baltimore  \
0    Cust1         1       1             0                1          0   
1    Cust2         0       1             1                1          1   
2    Cust3         1       1             1                1          1   
3    Cust4         0       0             0                1          1   
4    Cust5         1       1             0                1          0   

   Richmond  Virginia Beach  Charlotte  Raleigh  Atlanta  Nashville  
0         0               0          1        0        1          0  
1         0               0          0        1

<ipython-input-1-c0ceebdff40f>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  location_to_customer_cost[numeric_cols] = location_to_customer_cost[numeric_cols].applymap(clean_numeric)


In [2]:
import pulp

In [ ]:
# Binary decision variable y_ij: 1 if customer i is served by facility j, 0 otherwise
# Since a_ij is a nested dictionary, we iterate over the customers and locations within a_ij


In [ ]:
#y_ij

In [3]:
import pulp

# Step 1: Initialize the LP problem
problem = pulp.LpProblem("Facility_Location_Problem", pulp.LpMinimize)

# Step 2: Define Decision Variables
# Binary decision variable x_jk: 1 if a facility j is built at scale k, 0 otherwise
x_jk = pulp.LpVariable.dicts("x_jk", ((loc, scale) for loc, scale in f_jk), cat='Binary')

# Binary decision variable y_ij: 1 if customer i is served by facility j, 0 otherwise
y_ij = pulp.LpVariable.dicts("y_ij", ((cust, loc) for cust in a_ij for loc in a_ij[cust]), cat='Binary')

# Engineer hosting variable w_j (number of engineers assigned to location j)
w_j = pulp.LpVariable.dicts("w_j", (loc for loc in c_j), lowBound=0, cat='Integer')

# Step 3: Define the Objective Function (Minimize total cost)
# Objective function is the sum of office rent, traveling cost, and engineer hosting cost

# 1. Office rent cost: sum(f_jk * x_jk)
office_rent_cost = pulp.lpSum([x_jk[(loc, scale)] * f_jk[(loc, scale)] for loc, scale in f_jk])

# 2. Traveling cost: sum(h_i * d_ij * y_ij)
traveling_cost = pulp.lpSum([h_i[cust] * d_ij[cust][loc] * y_ij[(cust, loc)]
                             for cust in h_i for loc in d_ij[cust]])

# 3. Engineer hosting cost: sum(c_j * w_j)
engineer_hosting_cost = pulp.lpSum([c_j[loc] * w_j[loc] for loc in c_j])

# Total cost (objective function)
total_cost = office_rent_cost + traveling_cost + engineer_hosting_cost

# Add the objective function to the problem
problem += total_cost, "Minimize Total Cost"

# You can now add constraints (if any) and solve the problem

In [10]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 85.9 MB/s eta 0:00:00


In [4]:
# Add the constraints to the problem

# Constraint 1: At most one scale level for each location
for loc in c_j:
    problem += pulp.lpSum([x_jk[(loc, scale)] for scale in ['Small', 'Medium', 'Large']]) <= 1, f"Max_One_Scale_{loc}"

# Constraint 2: Only a built facility may serve customers
for cust in h_i:
    for loc in a_ij[cust]:
        problem += y_ij[(cust, loc)] <= pulp.lpSum([x_jk[(loc, scale)] for scale in ['Small', 'Medium', 'Large']]), f"Serve_Only_Built_{cust}_{loc}"

# Constraint 3: Each customer must be served by one facility
for cust in h_i:
    problem += pulp.lpSum([a_ij[cust][loc] * y_ij[(cust, loc)] for loc in a_ij[cust]]) == 1, f"Serve_Customer_{cust}"

# Ensures that the number of engineers allocated to a facility does not exceed its capacity based on the chosen scale.
for loc in m_jk:
    problem += w_j[loc] <= pulp.lpSum([m_jk[loc][scale] * x_jk[(loc, scale)] for scale in ['Small', 'Medium', 'Large']]), f"Capacity_Constraint_{loc}"

# Ensures that the allocated engineers can fulfill the service demands assigned to a facility.
for loc in c_j:
    problem += s * w_j[loc] >= pulp.lpSum([h_i[cust] * y_ij[(cust, loc)] for cust in h_i]), f"Service_Requirement_Constraint_{loc}"




In [ ]:
m_jk['New York']['Small']

50

In [5]:
# Solve the problem
problem.solve()

1

In [ ]:
x_jk[('New York','Small')].varValue

0.0

In [6]:
import pandas as pd

# List to store results for each location
results = []

# Loop over each location and gather data
for loc in w_j:
    engineers_allocated = int(w_j[loc].varValue) if w_j[loc].varValue is not None else 0
    customers_assigned = sum([y_ij[(cust, loc)].varValue for cust in h_i if (cust, loc) in y_ij])
    services_assigned = sum([h_i[cust] * y_ij[(cust, loc)].varValue for cust in h_i if (cust, loc) in y_ij])

    # Find the chosen scale for each location
    chosen_scale = "None"  # Default value
    for scale in ['Small', 'Medium', 'Large']:
        if (loc, scale) in x_jk and x_jk[(loc, scale)].varValue == 1:
            chosen_scale = scale.capitalize()
            break

    # Append results to the list
    results.append([loc, chosen_scale, engineers_allocated, int(customers_assigned), int(services_assigned)])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=["Location", "Scale", "Engineers Allocated", "Customers Assigned", "Services Assigned"])

# Calculate totals and create a DataFrame for the totals row
totals = pd.DataFrame({
    "Location": ["Total"],
    "Scale": [f"Large {sum(results_df['Scale'] == 'Large')}, Small {sum(results_df['Scale'] == 'Small')}"],
    "Engineers Allocated": [results_df["Engineers Allocated"].sum()],
    "Customers Assigned": [results_df["Customers Assigned"].sum()],
    "Services Assigned": [results_df["Services Assigned"].sum()]
})

# Concatenate the totals row with the original DataFrame
results_df = pd.concat([results_df, totals], ignore_index=True)

# Print the results in table form
print(results_df)


           Location             Scale  Engineers Allocated  \
0          New York              None                    0   
1            Boston              None                    0   
2      Philadelphia              None                    0   
3   Washington D.C.              None                    0   
4         Baltimore              None                    0   
5          Richmond             Small                    7   
6    Virginia Beach              None                    0   
7         Charlotte            Medium                    6   
8           Raleigh            Medium                    7   
9           Atlanta              None                    0   
10        Nashville              None                    0   
11            Total  Large 0, Small 1                   20   

    Customers Assigned  Services Assigned  
0                    0                  0  
1                    0                  0  
2                    0                  0  
3                

In [7]:
results_df

,Location,Scale,Engineers Allocated,Customers Assigned,Services Assigned
0,New York,None,0,0,0
1,Boston,None,0,0,0
2,Philadelphia,None,0,0,0
3,Washington D.C.,None,0,0,0
4,Baltimore,None,0,0,0
5,Richmond,Small,7,16,1381
6,Virginia Beach,None,0,0,0
7,Charlotte,Medium,6,14,1166
8,Raleigh,Medium,7,20,1392
9,Atlanta,None,0,0,0


from matplotlib import pyplot as plt
results_df['Engineers Allocated'].plot(kind='hist', bins=20, title='Engineers Allocated')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df['Customers Assigned'].plot(kind='hist', bins=20, title='Customers Assigned')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df['Services Assigned'].plot(kind='hist', bins=20, title='Services Assigned')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
results_df.groupby('Scale').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Scale'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='Engineers Allocated', y='Scale', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Scale'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='Customers Assigned', y='Scale', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Scale'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='Services Assigned', y='Scale', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)